Soft Actor-Critic
=================

(Previously: `Background for TD3`)

Soft Actor Critic (SAC) is an algorithm that optimizes a stochastic policy in an off-policy way, <font color=red>forming a bridge between stochastic policy optimization and DDPG-style approaches</font>. It isn't a direct successor to TD3 (having been published roughly concurrently), but it incorporates the clipped double-Q trick, and due to the inherent stochasticity of the policy in SAC, it also winds up benefiting from something like target policy smoothing. 

A central feature of SAC is **entropy regularization.** <font color=red>The policy is trained to maximize a trade-off between expected return and `entropy`, a measure of randomness in the policy</font>. This has a close connection to the exploration-exploitation trade-off: increasing entropy results in more exploration, which can accelerate learning later on. It can also prevent the policy from prematurely converging to a bad local optimum. 

.. _`entropy`: https://en.wikipedia.org/wiki/Entropy_(information_theory)

Quick Facts
-----------

* SAC is an off-policy algorithm.
* The version of SAC implemented here can only be used for environments with continuous action spaces.
* An alternate version of SAC, which slightly changes the policy update rule, can be implemented to handle discrete action spaces.
* The Spinning Up implementation of SAC does not support parallelization.

Key Equations
-------------

To explain Soft Actor Critic, we first have to introduce the entropy-regularized reinforcement learning setting. In entropy-regularized RL, there are slightly-different equations for value functions. 

### Entropy-Regularized Reinforcement Learning

Entropy is a quantity which, roughly speaking, says how random a random variable is. If a coin is weighted so that it almost always comes up heads, it has low entropy; if it's evenly weighted and has a half chance of either outcome, it has high entropy. 

Let $x$ be a random variable with probability mass or density function $P$. The entropy $H$ of $x$ is computed from its distribution $P$ according to

\begin{align*}
    H(P) = \underset{x \sim P}{E}{[-\log P(x)]}.
\end{align*}

In entropy-regularized reinforcement learning, the agent gets a bonus reward at each time step proportional to the entropy of the policy at that timestep. This changes `the RL problem` to:

\begin{align*}
    \pi^* = \arg \max_{\pi} E_{\tau \sim \pi}\left[
    { \sum_{t=0}^{\infty} \gamma^t \bigg( R(s_t, a_t, s_{t+1}) + \alpha H\left(\pi(\cdot|s_t)\right) \bigg)}\right],
\end{align*}

where $\alpha > 0$ is the trade-off coefficient. (Note: we're assuming an infinite-horizon discounted setting here, and we'll do the same for the rest of this page.) We can now define the slightly-different value functions in this setting. $V^{\pi}$ is changed to include the entropy bonuses from every timestep:

\begin{align*}
    V^{\pi}(s) = E_{\tau \sim \pi}{\left[\left. \sum_{t=0}^{\infty} \gamma^t \bigg( R(s_t, a_t, s_{t+1}) + \alpha H\left(\pi(\cdot|s_t)\right) \bigg) \right| s_0 = s\right]}
\end{align*}

$Q^{\pi}$ is changed to include the entropy bonuses from every timestep *except the first*:

\begin{align*}
    Q^{\pi}(s,a) = E_{\tau \sim \pi}\left[{ \left. \sum_{t=0}^{\infty} \gamma^t  R(s_t, a_t, s_{t+1}) + \alpha \sum_{t=1}^{\infty} \gamma^t H\left(\pi(\cdot|s_t)\right)\right| s_0 = s, a_0 = a}\right]
\end{align*}

With these definitions, $V^{\pi}$ and $Q^{\pi}$ are connected by:

\begin{align*}
    V^{\pi}(s) = E_{a \sim \pi}\left[{Q^{\pi}(s,a)} + \alpha H\left(\pi(\cdot|s)\right)\right]
\end{align*}

and the Bellman equation for $Q^{\pi}$ is

\begin{align*}
    Q^{\pi}(s,a) &= E_{s' \sim P \\ a' \sim \pi}\left[{R(s,a,s') + \gamma\left(Q^{\pi}(s',a') + \alpha H\left(\pi(\cdot|s')\right) \right)}\right] \\
    &= E_{s' \sim P}\left[{R(s,a,s') + \gamma V^{\pi}(s')}\right].
\end{align*}

.. `the RL problem`: ../spinningup/rl_intro.html#the-rl-problem

**.. admonition:: You Should Know**

The way we've set up the value functions in the entropy-regularized setting is a little bit arbitrary, and actually we could have done it differently (eg make $Q^{\pi}$ include the entropy bonus at the first timestep). The choice of definition may vary slightly across papers on the subject.


### Soft Actor-Critic


SAC concurrently learns a policy $\pi_{\theta}$ and two Q-functions $Q_{\phi_1}, Q_{\phi_2}$. There are two variants of SAC that are currently standard: one that uses a fixed entropy regularization coefficient $\alpha$, and another that enforces an entropy constraint by varying $\alpha$ over the course of training. For simplicity, Spinning Up makes use of the version with a fixed entropy regularization coefficient, but the entropy-constrained variant is generally preferred by practitioners.

**.. admonition:: You Should Know**

The SAC algorithm has changed a little bit over time. An older version of SAC also learns a value function $V_{\psi}$ in addition to the Q-functions; this page will focus on the modern version that omits the extra value function.



**Learning Q.** The Q-functions are learned in a similar way to TD3, but with a few key differences. 

**First, what's similar?**

1) Like in TD3, both Q-functions are learned with MSBE minimization, by regressing to a single shared target.

2) Like in TD3, the shared target is computed using target Q-networks, and the target Q-networks are obtained by polyak averaging the Q-network parameters over the course of training.

3) Like in TD3, the shared target makes use of the **clipped double-Q** trick.

**What's different?**

1) Unlike in TD3, the target also includes a term that comes from SAC's use of entropy regularization.

2) Unlike in TD3, <font color = red>the next-state actions used in the target come from the **current policy** instead of a target policy</font>.

3) Unlike in TD3, there is no explicit target policy smoothing. TD3 trains a deterministic policy, and so it accomplishes smoothing by adding random noise to the next-state actions. SAC trains a stochastic policy, and so the noise from that stochasticity is sufficient to get a similar effect.

Before we give the final form of the Q-loss, let’s take a moment to discuss how the contribution from entropy regularization comes in. We'll start by taking our recursive Bellman equation for the entropy-regularized $Q^{\pi}$ from earlier, and rewriting it a little bit by using the definition of entropy:

\begin{align*}
    Q^{\pi}(s,a) &= E_{s' \sim P \\ a' \sim \pi}\left[{R(s,a,s') + \gamma\left(Q^{\pi}(s',a') + \alpha H\left(\pi(\cdot|s')\right) \right)}\right] \\
    &= E_{s' \sim P \\ a' \sim \pi}\left[{R(s,a,s') + \gamma\left(Q^{\pi}(s',a') - \alpha \log \pi(a'|s') \right)}\right] 
\end{align*}

The RHS is an expectation over next states (which come from the replay buffer) and next actions (which come from the current policy, and **not** the replay buffer). Since it's an expectation, we can approximate it with samples:

\begin{align*}
    Q^{\pi}(s,a) &\approx r + \gamma\left(Q^{\pi}(s',\tilde{a}') - \alpha \log \pi(\tilde{a}'|s') \right), \;\;\;\;\;  \tilde{a}' \sim \pi(\cdot|s').
\end{align*}

**.. admonition:: You Should Know**

We switch next action notation to $\tilde{a}'$, instead of $a'$, to highlight that the next actions have to be sampled fresh from the policy (whereas by contrast, $r$ and $s'$ should come from the replay buffer).

SAC sets up the MSBE loss for each Q-function using this kind of sample approximation for the target. The only thing still undetermined here is which Q-function gets used to compute the sample backup: like TD3, SAC uses the clipped double-Q trick, and takes the minimum Q-value between the two Q approximators. 

Putting it all together, the loss functions for the Q-networks in SAC are:

\begin{align*}
    L(\phi_i, {\mathcal D}) = \underset{(s,a,r,s',d) \sim {\mathcal D}}{{\mathrm E}}\left[
        \Bigg( Q_{\phi_i}(s,a) - y(r,s',d) \Bigg)^2
        \right],
\end{align*}

where the target is given by

\begin{align*}
    y(r, s', d) = r + \gamma (1 - d) \left( \min_{j=1,2} Q_{\phi_{\text{targ},j}}(s', \tilde{a}') - \alpha \log \pi_{\theta}(\tilde{a}'|s') \right), \;\;\;\;\; \tilde{a}' \sim \pi_{\theta}(\cdot|s').
\end{align*}

**Learning the Policy.** The policy should, in each state, act to maximize the expected future return plus expected future entropy. That is, it should maximize $V^{\pi}(s)$, which we expand out into

\begin{align*}
    V^{\pi}(s) &= E_{a \sim \pi}\left[{Q^{\pi}(s,a)} + \alpha H\left(\pi(\cdot|s)\right)\right]\\
    &= E_{a \sim \pi}\left[{Q^{\pi}(s,a) - \alpha \log \pi(a|s)}\right].
\end{align*}

The way we optimize the policy makes use of the **reparameterization trick**, in which <font color=red>a sample from $\pi_{\theta}(\cdot|s)$ is drawn by computing a deterministic function of state, policy parameters, and independent noise</font>. To illustrate: following the authors of the SAC paper, we use a squashed Gaussian policy, which means that samples are obtained according to

\begin{align*}
    \tilde{a}_{\theta}(s, \xi) = \tanh\left( \mu_{\theta}(s) + \sigma_{\theta}(s) \odot \xi \right), \;\;\;\;\; \xi \sim \mathcal{N}(0, I).
\end{align*}

**.. admonition:: You Should Know**

This policy has two key differences from the policies we use in the other policy optimization algorithms:

**1. The squashing function.** The $\tanh$ in the SAC policy ensures that actions are bounded to a finite range. This is absent in the VPG, TRPO, and PPO policies. It also changes the distribution: before the $\tanh$ the SAC policy is a factored Gaussian like the other algorithms' policies, but after the $\tanh$ it is not. (You can still compute the log-probabilities of actions in closed form, though: see the paper appendix for details.)

**2. The way standard deviations are parameterized.** In VPG, TRPO, and PPO, we represent the log std devs with state-independent parameter vectors. In SAC, we represent the log std devs as outputs from the neural network, meaning that they depend on state in a complex way. SAC with state-independent log std devs, in our experience, did not work. (Can you think of why? Or better yet: run an experiment to verify?)

The reparameterization trick allows us to rewrite the expectation over actions (which contains a pain point: the distribution depends on the policy parameters) into an expectation over noise (which removes the pain point: the distribution now has no dependence on parameters):

\begin{align*}
    E_{a \sim \pi_{\theta}}\left[{Q^{\pi_{\theta}}(s,a) - \alpha \log \pi_{\theta}(a|s)}\right] = E_{\xi \sim \mathcal{N}}\left[{Q^{\pi_{\theta}}(s,\tilde{a}_{\theta}(s,\xi)) - \alpha \log \pi_{\theta}(\tilde{a}_{\theta}(s,\xi)|s)}\right]
\end{align*}

To get the policy loss, the final step is that we need to substitute $Q^{\pi_{\theta}}$ with one of our function approximators. Unlike in TD3, which uses $Q_{\phi_1}$ (just the first Q approximator), SAC uses $\min_{j=1,2} Q_{\phi_j}$ (the minimum of the two Q approximators). The policy is thus optimized according to

\begin{align*}
    \max_{\theta} E_{s \sim \mathcal{D} \\ \xi \sim \mathcal{N}}\left[{\min_{j=1,2} Q_{\phi_j}(s,\tilde{a}_{\theta}(s,\xi)) - \alpha \log \pi_{\theta}(\tilde{a}_{\theta}(s,\xi)|s)}\right],
\end{align*}

which is almost the same as the DDPG and TD3 policy optimization, except for the min-double-Q trick, the stochasticity, and the entropy term.


Exploration vs. Exploitation
----------------------------

SAC trains a stochastic policy with entropy regularization, and explores in an on-policy way. The entropy regularization coefficient $\alpha$ explicitly controls the explore-exploit tradeoff, with higher $\alpha$ corresponding to more exploration, and lower $\alpha$ corresponding to more exploitation. The right coefficient (the one which leads to the stablest / highest-reward learning) may vary from environment to environment, and could require careful tuning.

At test time, to see how well the policy exploits what it has learned, we remove stochasticity and use the mean action instead of a sample from the distribution. This tends to improve performance over the original stochastic policy.

**.. admonition:: You Should Know**

Our SAC implementation uses a trick to improve exploration at the start of training. For a fixed number of steps at the beginning (set with the ``start_steps`` keyword argument), the agent takes actions which are sampled from a uniform random distribution over valid actions. After that, it returns to normal SAC exploration.


Pseudocode
----------
![title](https://spinningup.openai.com/en/latest/_images/math/c01f4994ae4aacf299a6b3ceceedfe0a14d4b874.svg)

In [7]:
print('who knows')

who knows
